<h1>Assignment-2 : Text Classification</h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import sys
import re


2025-02-20 15:15:55.599385: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-20 15:15:55.623380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740044755.654055   23476 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740044755.662793   23476 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 15:15:55.686962: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

import csv
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
csv.field_size_limit(sys.maxsize)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vishal27/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/vishal27/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/vishal27/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


131072

In [ ]:
# Load data from Dataset-1.xlsx file 
data = pd.read_excel('Dataset-1.xlsx')

# Preprocess data

def preprocess_data(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize sentences and words
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]
    
    # Remove stop words
    words = [[word for word in sentence if word not in stop_words] for sentence in words]
    
    # Stem words
    words = [[ps.stem(word) for word in sentence] for sentence in words]
    
    # Lemmatize words
    words = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in words]
    
    # Join words back into sentences
    text =''.join([' '.join(sentence) for sentence in words])
    
    return text



In [ ]:
#data in the Dataset-1.xlsx file is in format : ID	TITLE	ABSTRACT	Computer Science	Physics	Mathematics	Statistics	Quantitative Biology	Quantitative Finance
# Computer Science, Physics, Mathematics, Statistics, Quantitative Biology, Quantitative Finance are fields and one of them is marked 1 and rest 0
# Abstract is input and the fiels are the output
